In [1]:
import wrds
import pandas as pd
from datetime import datetime
# import ray
# import os

# # Set environment variables for Modin and Ray
# os.environ["MODIN_ENGINE"] = "ray"
# os.environ["__MODIN_AUTOIMPORT_PANDAS__"] = "1"
# # Look at the Ray documentation with respect to the Ray configuration suited to you most.
# ray.init()

def fetch_prices_for_dates(cusip_list, date_list):
    print("Fetching")
    cusip_list_str = "', '".join(cusip_list)
    date_list_str = "', '".join(pd.to_datetime(date_list).strftime('%Y-%m-%d'))
    
    query = f"""
        SELECT 
            cusip,
            datadate,
            ajexdi,
            prccd,
            trfd
        FROM 
            comp_na_daily_all.secd
        WHERE 
            cusip IN ('{cusip_list_str}') AND
            datadate IN ('{date_list_str}')
    """
    return db.raw_sql(query)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def first_non_na(row, columns):
    for col in columns:
        if pd.notna(row[col]):
            return row[col]
    return None

def downcast_numeric_columns(df):
    """
    Downcast numerical columns in a pandas DataFrame to reduce memory usage.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame to downcast.
    
    Returns:
    pd.DataFrame: DataFrame with downcasted numeric columns.
    """
    # Downcast integer columns
    int_cols = df.select_dtypes(include=['int', 'int64']).columns
    df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')
    
    # Downcast float columns
    float_cols = df.select_dtypes(include=['float', 'float64']).columns
    df[float_cols] = df[float_cols].apply(pd.to_numeric, downcast='float')
    
    return df


# Establish a connection to the WRDS database
db = wrds.Connection(wrds_username='asherbaraban')

Loading library list...
Done


In [2]:
insiders_df = pd.read_pickle("raw_insiders_pre_merge.pkl")
# cusip_list = insiders_df['cusipi'].unique().tolist()
# trandate_list = insiders_df['trandate'].unique().tolist()

# # Generate the additional dates
# trandate_6mo_list = (insiders_df['trandate'] + pd.DateOffset(months=6)).unique().tolist()
# trandate_6mo_1d_list = (insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=1)).unique().tolist()
# trandate_6mo_2d_list = (insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=2)).unique().tolist()

# # Combine all date lists
# combined_date_list = list(set(trandate_list + trandate_6mo_list + trandate_6mo_1d_list + trandate_6mo_2d_list))
# print(len(combined_date_list))
# start_time = datetime.now()
# print(start_time)
# final_prices_df = pd.DataFrame()
# date_chunks = list(chunks(combined_date_list, 3000))

# for i, date_chunk in enumerate(date_chunks):
#     db = wrds.Connection(wrds_username='asherbaraban')
#     chunk_start_time = datetime.now()
#     print(f"Processing chunk {i+1}/{len(date_chunks)}")

#     chunk_prices_df = fetch_prices_for_dates(cusip_list, date_chunk)
#     chunk_prices_df['datadate'] = pd.to_datetime(chunk_prices_df['datadate'])
#     final_prices_df = pd.concat([final_prices_df, chunk_prices_df], ignore_index=True)

#     chunk_end_time = datetime.now()
#     print(f"Chunk {i+1} processed in: {chunk_end_time - chunk_start_time}")
#     db.close()

# end_time = datetime.now()
# print(f"Data fetched in: {end_time - start_time}")

# # Verify the concatenated DataFrame
# print(f"Total rows fetched: {len(final_prices_df)}")
# print(final_prices_df.head())

In [3]:
# final_prices_df.to_csv("raw_prices.csv", index=False)
# final_prices_df.to_pickle('raw_prices.pkl')


In [4]:
# final_prices_df.info()

In [5]:
final_prices_df = pd.read_pickle("raw_prices.pkl")
final_prices_df = downcast_numeric_columns(final_prices_df)
insiders_df = downcast_numeric_columns(insiders_df)

In [6]:
insiders_df['trandate_6mo'] = insiders_df['trandate'] + pd.DateOffset(months=6)
insiders_df['trandate_6mo_1'] = insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=1)
insiders_df['trandate_6mo_2'] = insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=2)

In [7]:
insiders_df = insiders_df.merge(final_prices_df,
                                left_on=['cusipi', 'trandate'],
                                right_on=['cusip', 'datadate'],
                                how='left')

In [8]:
insiders_df = insiders_df.merge(final_prices_df,
                                left_on=['cusipi', 'trandate_6mo'],
                                right_on=['cusip', 'datadate'],
                                how='left',
                                suffixes=('', '_6mo'))
insiders_df = insiders_df.merge(final_prices_df,
                                left_on=['cusipi', 'trandate_6mo_1'],
                                right_on=['cusip', 'datadate'],
                                how='left',
                                suffixes=('', '_6mo_1'))
insiders_df = insiders_df.merge(final_prices_df,
                                left_on=['cusipi', 'trandate_6mo_2'],
                                right_on=['cusip', 'datadate'],
                                how='left',
                                suffixes=('', '_6mo_2'))

insiders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245190 entries, 0 to 9245189
Data columns (total 41 columns):
 #   Column          Dtype         
---  ------          -----         
 0   dcn             string        
 1   seqnum          int16         
 2   personid        int32         
 3   owner           category      
 4   rolecode1       category      
 5   rolecode2       category      
 6   rolecode3       category      
 7   rolecode4       category      
 8   cname           category      
 9   ticker          category      
 10  sector          category      
 11  industry        object        
 12  trandate        datetime64[ns]
 13  tprice          float64       
 14  ownership       category      
 15  cleanse         category      
 16  acqdisp         category      
 17  cusipi          object        
 18  trandate_6mo    datetime64[ns]
 19  trandate_6mo_1  datetime64[ns]
 20  trandate_6mo_2  datetime64[ns]
 21  cusip           object        
 22  datadate        da

In [9]:
# List of columns to check in order of priority
price_columns = ['prccd_6mo', 'prccd_6mo_1', 'prccd_6mo_2']
adjustment_columns = ['ajexdi_6mo', 'ajexdi_6mo_1', 'ajexdi_6mo_2']
total_return_columns = ['trfd_6mo', 'trfd_6mo_1', 'trfd_6mo_2']

# Apply the function to each row to get the first non-NA value
insiders_df['prccd_6mo_consolidated'] = insiders_df.apply(lambda row: first_non_na(row, price_columns), axis=1)
insiders_df['ajexdi_6mo_consolidated'] = insiders_df.apply(lambda row: first_non_na(row, adjustment_columns), axis=1)
insiders_df['trfd_6mo_consolidated'] = insiders_df.apply(lambda row: first_non_na(row, total_return_columns), axis=1)

In [10]:
insiders_df['prccd_adjusted_begin'] = insiders_df['prccd'] / insiders_df['ajexdi']
insiders_df['prccd_adjusted_end'] = insiders_df['prccd_6mo_consolidated'] / insiders_df['ajexdi_6mo_consolidated']

insiders_df['total_return_6mo'] = (
    (insiders_df['prccd_adjusted_end'] * insiders_df['trfd_6mo_consolidated']) /
    (insiders_df['prccd_adjusted_begin'] * insiders_df['trfd'])
) - 1

insiders_df.info()
insiders_df.to_pickle("merged_prices_insiders.pkl")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245190 entries, 0 to 9245189
Data columns (total 47 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   dcn                      string        
 1   seqnum                   int16         
 2   personid                 int32         
 3   owner                    category      
 4   rolecode1                category      
 5   rolecode2                category      
 6   rolecode3                category      
 7   rolecode4                category      
 8   cname                    category      
 9   ticker                   category      
 10  sector                   category      
 11  industry                 object        
 12  trandate                 datetime64[ns]
 13  tprice                   float64       
 14  ownership                category      
 15  cleanse                  category      
 16  acqdisp                  category      
 17  cusipi                   ob

In [11]:
# insiders_df.to_pickle("merged_prices_insiders.pkl")

In [12]:
insiders_df = pd.read_pickle("merged_prices_insiders.pkl")
insiders_df['trandate'] = pd.to_datetime(insiders_df['trandate'])
insiders_df['trandate_6mo'] = pd.to_datetime(insiders_df['trandate_6mo'])
insiders_df['trandate_6mo_1'] = pd.to_datetime(insiders_df['trandate_6mo_1'])
insiders_df['trandate_6mo_2'] = pd.to_datetime(insiders_df['trandate_6mo_2'])
insiders_df = downcast_numeric_columns(insiders_df)

object_cols = ['owner', 'rolecode1', 'rolecode2', 'rolecode3', 'rolecode4', 'cname', 'ticker', 'sector', 'ownership', 'cleanse', 'acqdisp', 'cusipi']
for col in object_cols:
    insiders_df[col] = insiders_df[col].astype('category')
insiders_df['dcn'] = insiders_df['dcn'].astype('string')
insiders_df['seqnum'] = pd.to_numeric(insiders_df['seqnum'], downcast='integer')
insiders_df['personid'] = pd.to_numeric(insiders_df['personid'], downcast='integer')

db = wrds.Connection(wrds_username='asherbaraban')
sector_etfs = {
    1: "XLF",  # Finance
    2: "XLV",  # Healthcare
    3: "IYK",  # Consumer Non-Durable
    4: "XLY",  # Consumer Services
    5: "XHB",  # Consumer Durables
    6: "XLE",  # Energy
    7: "IYT",  # Transportation
    8: "XLK",  # Technology
    9: "XLB",  # Basic Industries
    10: "XLI",  # Capital Goods
    11: "XLU",  # Public Utilities
    99: "VTI",  # Miscellaneous
    0: "VTI",  # Not Classified
    "XX": "VTI",  # Not Classified
}

benchmark_tickers = sector_etfs.values()
tickers_placeholder = ', '.join(f"'{ticker}'" for ticker in benchmark_tickers)

query = f"""
        SELECT 
            cusip,
            datadate,
            ajexdi,
            trfd,
            prccd,
            tic
        FROM 
            comp_na_daily_all.secd
        WHERE 
            tic IN ({tickers_placeholder}) AND
            datadate = '2023-10-02'
    """
prices = db.raw_sql(query)

ticker_to_cusip = prices.set_index('tic')['cusip'].to_dict()
# Generate the additional dates
trandate_list = insiders_df['trandate'].unique().tolist()
trandate_6mo_list = (insiders_df['trandate'] + pd.DateOffset(months=6)).unique().tolist()
trandate_6mo_1d_list = (insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=1)).unique().tolist()
trandate_6mo_2d_list = (insiders_df['trandate'] + pd.DateOffset(months=6) +  pd.DateOffset(days=2)).unique().tolist()

# Combine all date lists
combined_date_list = list(set(trandate_list + trandate_6mo_list + trandate_6mo_1d_list + trandate_6mo_2d_list))

# Now query the right dates to get total returns for all of the right intervals 
etf_prices = fetch_prices_for_dates(ticker_to_cusip.values(), combined_date_list)
etf_prices['datadate'] = pd.to_datetime(insiders_df['datadate'])
insiders_df['sector_ticker'] = insiders_df['sector'].map(sector_etfs)
insiders_df['sector_cusip'] = insiders_df['sector_ticker'].map(ticker_to_cusip)
etf_prices['cusip'] = etf_prices['cusip'].astype('category')
etf_prices.dropna(subset=['datadate'], inplace=True)

etf_prices['price_adj'] = (etf_prices['prccd'] * etf_prices['trfd']) / etf_prices['ajexdi']
etf_prices.drop(columns = ['prccd', 'trfd', 'ajexdi'], inplace=True)

etf_prices.rename(columns={'datadate': 'date', 'cusip': 'sector_cusip'}, inplace=True)
etf_prices.set_index(['date', 'sector_cusip'], inplace=True)
etf_prices.sort_index(inplace=True)
etf_prices.info()

Loading library list...
Done
Fetching
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 60632 entries, (Timestamp('1988-11-16 00:00:00'), '81369Y100') to (Timestamp('2003-06-11 00:00:00'), '81369Y803')
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price_adj  60632 non-null  float64
dtypes: float64(1)
memory usage: 691.9 KB


In [13]:
insiders_df['sector_cusip'] = insiders_df['sector_cusip'].astype('category')
insiders_df['sector_ticker'] = insiders_df['sector_ticker'].astype('category')
insiders_df['industry'] = insiders_df['industry'].astype('category')
insiders_df['cusip_6mo'] = insiders_df['cusip_6mo'].astype('string')

substrings = ['trfd', 'ajexdi', 'prccd', 'datadate', 'cusip_', 'sector_ticker', 'industry', 'cname', 'tprice']

# Identify columns to drop based on the substrings
columns_to_drop = [col for col in insiders_df.columns if any(sub in col for sub in substrings)]

# Drop the identified columns inplace
insiders_df.drop(columns=columns_to_drop, inplace=True)
insiders_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245190 entries, 0 to 9245189
Data columns (total 21 columns):
 #   Column            Dtype         
---  ------            -----         
 0   dcn               string        
 1   seqnum            int16         
 2   personid          int32         
 3   owner             category      
 4   rolecode1         category      
 5   rolecode2         category      
 6   rolecode3         category      
 7   rolecode4         category      
 8   ticker            category      
 9   sector            category      
 10  trandate          datetime64[ns]
 11  ownership         category      
 12  cleanse           category      
 13  acqdisp           category      
 14  cusipi            category      
 15  trandate_6mo      datetime64[ns]
 16  trandate_6mo_1    datetime64[ns]
 17  trandate_6mo_2    datetime64[ns]
 18  cusip             object        
 19  total_return_6mo  float32       
 20  sector_cusip      category      
dtypes: categ

In [14]:
# Merge with trandate_6mo
insiders_df.rename(columns={'trandate': 'date'}, inplace=True)
print(insiders_df.columns)
insiders_df.set_index(['date', 'sector_cusip'], inplace=True)
insiders_df = insiders_df.merge(etf_prices, left_index=True, right_index=True, how='left', suffixes=('', '_sec'))
insiders_df.reset_index(inplace=True)
insiders_df.rename(columns={'date': 'trandate'}, inplace=True)
print("A completed")

# Merge with trandate_6mo
insiders_df.rename(columns={'trandate_6mo': 'date'}, inplace=True)
insiders_df.set_index(['date', 'sector_cusip'], inplace=True)
insiders_df = insiders_df.merge(etf_prices, left_index=True, right_index=True, how='left', suffixes=('', '_sec_6mo'))
insiders_df.reset_index(inplace=True)
insiders_df.rename(columns={'date': 'trandate_6mo'}, inplace=True)
print("B completed")

# Merge with trandate_6mo_1
insiders_df.rename(columns={'trandate_6mo_1': 'date'}, inplace=True)
insiders_df.set_index(['date', 'sector_cusip'], inplace=True)
insiders_df = insiders_df.merge(etf_prices, left_index=True, right_index=True, how='left', suffixes=('', '_sec_6mo_1'))
insiders_df.reset_index(inplace=True)
insiders_df.rename(columns={'date': 'trandate_6mo_1'}, inplace=True)
print("C completed")

# Merge with trandate_6mo_2
insiders_df.rename(columns={'trandate_6mo_2': 'date'}, inplace=True)
insiders_df.set_index(['date', 'sector_cusip'], inplace=True)
insiders_df = insiders_df.merge(etf_prices, left_index=True, right_index=True, how='left', suffixes=('', '_sec_6mo_2'))
insiders_df.reset_index(inplace=True)
insiders_df.rename(columns={'date': 'trandate_6mo_2'}, inplace=True)
print("All merges completed")

# Save the merged dataframe
insiders_df.to_pickle("post_sector.pkl")

Index(['dcn', 'seqnum', 'personid', 'owner', 'rolecode1', 'rolecode2',
       'rolecode3', 'rolecode4', 'ticker', 'sector', 'date', 'ownership',
       'cleanse', 'acqdisp', 'cusipi', 'trandate_6mo', 'trandate_6mo_1',
       'trandate_6mo_2', 'cusip', 'total_return_6mo', 'sector_cusip'],
      dtype='object')
A completed
B completed
C completed
All merges completed


In [15]:
insiders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245190 entries, 0 to 9245189
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   trandate_6mo_2       datetime64[ns]
 1   sector_cusip         category      
 2   trandate_6mo_1       datetime64[ns]
 3   trandate_6mo         datetime64[ns]
 4   trandate             datetime64[ns]
 5   dcn                  string        
 6   seqnum               int16         
 7   personid             int32         
 8   owner                category      
 9   rolecode1            category      
 10  rolecode2            category      
 11  rolecode3            category      
 12  rolecode4            category      
 13  ticker               category      
 14  sector               category      
 15  ownership            category      
 16  cleanse              category      
 17  acqdisp              category      
 18  cusipi               category      
 19  cusip                

In [16]:
xxx


import matplotlib.pyplot as plt
# Assuming insiders_df is already defined and has the column 'trandate'
insiders_df['trandate_year'] = insiders_df['trandate'].dt.year

# Group by the extracted year and calculate the mean total return
annual_return = insiders_df.groupby(['trandate_year', 'acqdisp'])['total_return_6mo'].mean().reset_index()
annual_return = annual_return[annual_return['acqdisp'] == 'A']
# Create a line plot of the average total return by year
plt.figure(figsize=(10, 6))
plt.plot(annual_return['trandate_year'], annual_return['total_return_6mo'], marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('Average Total Return (6 Months)')
plt.title('Average 6-Month Total Return by Year')
plt.grid(False)
plt.show()


NameError: name 'xxx' is not defined